In [1]:
from urllib.request import urlopen
import pandas as pd
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.12               |           py36_0         3.0 MB  conda-forge
    conda-package-handling-1.6.0|   py36h516909a_0         926 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py36h5169

In [45]:

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [2]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::scikit-learn==0.19.2=py37h4989274_0
  - defaults/linux-64::anaconda==5.3.0=py37_0
  - defaults/linux-64::numba==0.40.0=py36h962f231_0
  - defaults/linux-64::blaze==0.11.3=py36_0
  - defaults/linux-64::scikit-image==0.14.0=py37hf484d3e_1
done

## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py36_0           5 KB
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    anaconda-custom            |           py36_1           3 KB
    backports.os-0.1.1         |        py36_1001          15 KB  conda-forge
    branca-0.3.1               |  

In [3]:
#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(wiki) 
soup = BeautifulSoup(page)
canda_table=soup.find('table', class_="wikitable sortable")
Postcode=[]
Borough=[]
Neighbourhood=[]
for row in canda_table.findAll("tr"):
    row_cells=row.find_all('td')
    if len(row_cells)==3:
        Postcode.append(row_cells[0].find(text=True))
        Borough.append(row_cells[1].find(text=True))
        Neighbourhood.append(row_cells[2].find(text=True))


df=pd.DataFrame()
df['Postcode']=Postcode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood
assigned_Borough=df[df.Borough!='Not assigned']
after_Merge = assigned_Borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
after_Merge=after_Merge.replace('Not assigned\n',after_Merge[after_Merge.Neighbourhood.str.contains('Not assigned\n')].Borough.to_string())
after_Merge.rename(columns={'Postcode':'Postal Code'},inplace=True)
after_Merge.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


<b> Using geocoder it is taking very long time so I have used the CSV file instead</b><br>
!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder


lat_lng_coords = None


while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M1B"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [4]:
latAndlongFromCSV=pd.read_csv("Geospatial_Coordinates.csv")
latAndlongFromCSV.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df3 = pd.merge(after_Merge, latAndlongFromCSV, how='outer', on=['Postal Code'])
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


In [6]:
df3.shape

(103, 5)

<b>Forming Data frame only with Borough Data contains word Toronto</b>

In [7]:
df_Torronto=df3[df3.Borough.str.contains("Toronto")]
df_Torronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West\n, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West\n, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District\n,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


<b> create map of Toronto neighborhoods with latitude and longitude values </b>

In [8]:
#43.651070
#-79.347015

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate ofToronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate ofToronto are 43.653963, -79.387207.


<b> Create a map of only Toronto with neighborhoods .</b>

In [14]:
# create map of Canada using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df_Torronto.Latitude, df_Torronto.Longitude, df_Torronto.Neighbourhood):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto




<b>Using Foursquare API will explore  all the neighborhoods of Tornoto and segment them.</b>

In [15]:
CLIENT_ID = 'OSYQN45NYOKZMCOWTR3X4QAAWSBQKVG3UE0DMSSCPEVF0RNF' # your Foursquare ID
CLIENT_SECRET = 'XW2SOPF0P1PB1EWD0FZDBEESOVYNF2TMG5MNRWLQPNLIXFX0' # your Foursquare Secret
VERSION = '20190926' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OSYQN45NYOKZMCOWTR3X4QAAWSBQKVG3UE0DMSSCPEVF0RNF
CLIENT_SECRET:XW2SOPF0P1PB1EWD0FZDBEESOVYNF2TMG5MNRWLQPNLIXFX0


<b>Lets Explore any one area in Tornoto and the famous venues and trendings in and around that area</b>

In [42]:
NeighbourHood_name=df_Torronto.iloc[0].Neighbourhood
df3.Longitude

In [43]:
# type your answer here


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_Torronto.iloc[0].Latitude, 
    df_Torronto.iloc[0].Longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OSYQN45NYOKZMCOWTR3X4QAAWSBQKVG3UE0DMSSCPEVF0RNF&client_secret=XW2SOPF0P1PB1EWD0FZDBEESOVYNF2TMG5MNRWLQPNLIXFX0&v=20190926&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [46]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d8c6a4d475abd002cca5bfa'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

<b>Normalize the Json Data into DataFrame</b>

In [48]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bd461bc77b29c74a07d9282-0,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",4bd461bc77b29c74a07d9282,Glen Manor,CA,Toronto,Canada,Queen St.,...,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.67682094413784...",43.676821,-79.293942,NaN,ON,Glen Manor Ravine,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c062f964a52011f820e3-1,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",4ad4c062f964a52011f820e3,125 Southwood Dr,CA,Toronto,Canada,NaN,...,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'label': 'display', 'lat': 43.678879, 'lng':...",43.678879,-79.297734,M4E 0B8,ON,The Big Carrot Natural Food Market,0,[],75150878
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b8daea1f964a520480833e3-2,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",4b8daea1f964a520480833e3,676 Kingston Rd.,CA,Toronto,Canada,at Main St.,...,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'label': 'display', 'lat': 43.67918143494101...",43.679181,-79.297215,M4E 1R4,ON,Grover Pub and Grub,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4df91c4bae60f95f82229ad5-3,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",4df91c4bae60f95f82229ad5,NaN,CA,Toronto,Canada,NaN,...,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.68056321147582...",43.680563,-79.292869,NaN,ON,Upper Beaches,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4dbc53541e72b351caafda72-4,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",4dbc53541e72b351caafda72,629 Kingston Rd,CA,Toronto,Canada,at Main St,...,"[629 Kingston Rd (at Main St), Toronto ON, Can...","[{'label': 'display', 'lat': 43.67888788593709...",43.678888,-79.298167,NaN,ON,Seaspray Restaurant,0,[],NaN


<b>Drop unnencsary Columns</b>

In [49]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

<b> Format the Category</b>

In [51]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


TypeError: ('string indices must be integers', 'occurred at index 0')

In [52]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


<b> WE have explored the top venues in "The Beech " Neighbour hood. Now we will see all the venues for all the neighbour hoods in Tornot</b>

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Tornoto_Venues = getNearbyVenues(names=df_Torronto.Neighbourhood,
                                   latitudes=df_Torronto['Latitude'],
                                   longitudes=df_Torronto['Longitude']
                                  )


The Beaches
The Danforth West
, Riverdale
The Beaches West
, India Bazaar
Studio District

Lawrence Park
Davisville North

North Toronto West

Davisville

Moore Park, Summerhill East

Deer Park, Forest Hill SE
, Rathnelly, South Hill, Summerhill West

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson
, Garden District

St. James Town
Berczy Park
Central Bay Street

Adelaide
, King
, Richmond

Harbourfront East
, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel

Roselawn

Forest Hill North, Forest Hill West

The Annex, North Midtown
, Yorkville
Harbord
, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay
, Island airport
, Harbourfront West
, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade

First Canadian Place, Underground city
Christie

Dovercourt Village, Dufferin

Little Portugal, Trinity
Brockton
, Exhibition Place, Park

In [57]:
Tornoto_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide\n, King\n, Richmond\n",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton\n, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern\n,19,19,19,19,19,19
"CN Tower, Bathurst Quay\n, Island airport\n, Harbourfront West\n, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street\n,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie\n,16,16,16,16,16,16


In [58]:
print('There are {} uniques categories.'.format(len(Tornoto_Venues['Venue Category'].unique())))

There are 237 uniques categories.


In [59]:
Tornoto_Venues['Venue Category'].unique()

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Asian Restaurant', 'Greek Restaurant', 'Ice Cream Shop',
       'Cosmetics Shop', 'Italian Restaurant', 'Brewery', 'Yoga Studio',
       'Restaurant', 'Fruit & Vegetable Store', 'Dessert Shop',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Indian Restaurant',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Diner',
       'Bubble Tea Shop', 'Coffee Shop', 'Caribbean Restaurant', 'Bakery',
       'Sports Bar', 'Café', 'American Restaurant', 'Sushi Restaurant',
       'Liquor Store', 'Gym', 'Burger Joint', 'Fish & Chips Shop', 'Park',
       'Pet Store', 'Burrito Place', 'Steakhouse', 'Movie Theater',
       'Fast Food Restaurant', 'Sandwich Place', 'Fish Market', 'Gay Bar',
       'Cheese Shop', 'Chinese Restaurant', 'Middle Eastern Restaurant',
       'Thai Restaurant', 'Comfort Food Restaurant', 'Stationery Store',
       'Seafood Restaurant', 'Coworking Space', 'Gastropub',
       'Latin American Restaura

In [60]:
# one hot encoding
toronto_onehot = pd.get_dummies(Tornoto_Venues[['Venue Category']], prefix="", prefix_sep="")


In [62]:
toronto_onehot.head(20)

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Tornoto_Venues.Neighborhood
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]



In [71]:
toronto_onehot.head(20)

,Wine Bar,Wings Joint,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Wine Bar,Wings Joint,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,"Adelaide\n, King\n, Richmond\n",0.010000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.030000,0.010000,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.00000
1,Berczy Park,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.017544,0.017544,0.000000,0.00000,0.000000,0.000000,0.00,0.017544,0.000000,0.00000
2,"Brockton\n, Exhibition Place, Parkdale Village",0.000000,0.00000,0.043478,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
3,Business Reply Mail Processing Centre 969 East...,0.000000,0.00000,0.052632,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
4,"CN Tower, Bathurst Quay\n, Island airport\n, H...",0.000000,0.00000,0.000000,0.00000,0.066667,0.066667,0.133333,0.133333,0.133333,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
5,"Cabbagetown, St. James Town",0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.021739,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
6,Central Bay Street\n,0.011628,0.00000,0.011628,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011628,0.011628,0.000000,0.00000,0.000000,0.000000,0.00,0.011628,0.000000,0.00000
7,"Chinatown, Grange Park, Kensington Market",0.010000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.010000,0.000000,0.00000,0.010000,0.000000,0.00,0.060000,0.000000,0.04000
8,Christie\n,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
9,Church and Wellesley,0.000000,0.01087,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010870,0.010870,0.010870,0.01087,0.000000,0.000000,0.00,0.000000,0.010870,0.01087
